In [14]:

import math, json, os, sys

import keras
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.layers import Dense
from keras.models import Model
from keras.optimizers import Adam
from keras.preprocessing import image


print('Imports Done')

Imports Done


In [15]:
# paths to training and validation data

train_data_dir = 'dataset/train'
validation_data_dir = 'dataset/test'

In [4]:
# Params for CNN

# img_width, img_height = 150, 150
# batch_size = 5
# epochs = 200
# train_samples = 420
# validation_samples = 80
# img_channels = 3
# cardinality = 32

In [26]:
DATA_DIR = 'dataset'
TRAIN_DIR = os.path.join(DATA_DIR, 'train')
VALID_DIR = os.path.join(DATA_DIR, 'test')
SIZE = (224, 224)
BATCH_SIZE = 5

In [27]:
num_train_samples = sum([len(files) for r, d, files in os.walk(TRAIN_DIR)])
num_valid_samples = sum([len(files) for r, d, files in os.walk(VALID_DIR)])

In [28]:
num_train_steps = math.floor(num_train_samples/BATCH_SIZE)
num_valid_steps = math.floor(num_valid_samples/BATCH_SIZE)

In [29]:
gen = keras.preprocessing.image.ImageDataGenerator()
val_gen = keras.preprocessing.image.ImageDataGenerator(horizontal_flip=True, vertical_flip=True)

In [30]:
batches = gen.flow_from_directory(TRAIN_DIR, target_size=SIZE, class_mode='categorical', shuffle=True, batch_size=BATCH_SIZE)
val_batches = val_gen.flow_from_directory(VALID_DIR, target_size=SIZE, class_mode='categorical', shuffle=True, batch_size=BATCH_SIZE)

Found 420 images belonging to 2 classes.
Found 80 images belonging to 2 classes.


In [31]:
model = keras.applications.resnet50.ResNet50()

In [32]:
classes = list(iter(batches.class_indices))
model.layers.pop()
for layer in model.layers:
    layer.trainable=False
last = model.layers[-1].output
x = Dense(len(classes), activation="softmax")(last)
finetuned_model = Model(model.input, x)
finetuned_model.compile(optimizer=Adam(lr=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])
for c in batches.class_indices:
    classes[batches.class_indices[c]] = c
finetuned_model.classes = classes

In [33]:
early_stopping = EarlyStopping(patience=10)
checkpointer = ModelCheckpoint('resnet50_best.h5', verbose=1, save_best_only=True)

In [34]:
finetuned_model.fit_generator(batches, steps_per_epoch=num_train_steps, epochs=100, callbacks=[early_stopping, checkpointer], validation_data=val_batches, validation_steps=num_valid_steps)
finetuned_model.save('resnet50_final.h5')

Epoch 1/100
84/84 [==============================] - 27s 318ms/step - loss: 0.6677 - acc: 0.6119 - val_loss: 0.9808 - val_acc: 0.5375

Epoch 00001: val_loss improved from inf to 0.98081, saving model to resnet50_best.h5
Epoch 2/100
84/84 [==============================] - 8s 101ms/step - loss: 0.5437 - acc: 0.7286 - val_loss: 0.9552 - val_acc: 0.5625

Epoch 00002: val_loss improved from 0.98081 to 0.95516, saving model to resnet50_best.h5
Epoch 3/100
84/84 [==============================] - 9s 103ms/step - loss: 0.4540 - acc: 0.7833 - val_loss: 0.6280 - val_acc: 0.6500

Epoch 00003: val_loss improved from 0.95516 to 0.62803, saving model to resnet50_best.h5
Epoch 4/100
84/84 [==============================] - 8s 89ms/step - loss: 0.4231 - acc: 0.8024 - val_loss: 0.9071 - val_acc: 0.6125

Epoch 00004: val_loss did not improve
Epoch 5/100
84/84 [==============================] - 6s 69ms/step - loss: 0.4453 - acc: 0.7976 - val_loss: 0.8590 - val_acc: 0.6125

Epoch 00005: val_loss did not 